# Decentralized Machine Learning

In [143]:
import warnings
warnings.simplefilter('ignore')
import logging
import pickle
import shutil
import time

import numpy as np
import tensorflow as tf

from eth_utils import is_address
from web3.auto import w3
from web3 import Web3, HTTPProvider
# warnings.filterwarnings('ignore')

### Client Setup

In [144]:
from web3 import Web3, HTTPProvider


web3 = Web3(HTTPProvider('http://localhost:8545'))
# web3 = Web3(web3.currentProvider)

PASSPHRASE = 'panda'
TEST_ACCOUNT = web3.eth.coinbase

acct = web3.personal.newAccount(PASSPHRASE)
print("New Account:", acct)
print("Balance:", web3.eth.getBalance(acct))
web3.eth.sendTransaction({"from": TEST_ACCOUNT, "to": acct, "value": 9999999999})
print("Balance:", web3.eth.getBalance(acct))
# print("Accounts on the Network:", web3.eth.accounts)
# print("Personal Accounts:", web3.personal.listAccounts)

New Account: 0x924C78C99F98cBcB307Dd4a1Cf0118885282ca12
Balance: 0
Balance: 9999999999


### Contract Deployment

In [142]:
from solc import compile_source
import pprint


# CONTRACT_NAME = 'Greeter.sol'
CONTRACT_NAME = '../contracts/Query.sol'
ARG1 = 3
ARG2 = [0, 1, 2]

def compile_source_file(file_path):
   with open(file_path, 'r') as f:
      source = f.read()

   return compile_source(source)


def deploy_contract(w3, contract_interface, arg1, arg2):
    tx_hash = w3.eth.contract(
        abi=contract_interface['abi'],
        bytecode=contract_interface['bin']).constructor(3, [0,1,2], 0, 0, 5, [0,1,2], 3).transact({"from": acct})

    address = w3.eth.getTransactionReceipt(tx_hash)['contractAddress']
    return address


def wait_for_receipt(w3, tx_hash, poll_interval):
   while True:
       tx_receipt = w3.eth.getTransactionReceipt(tx_hash)
       if tx_receipt:
         return tx_receipt
       time.sleep(poll_interval)
    
contract_source_path = CONTRACT_NAME
compiled_sol = compile_source_file(contract_source_path)

pp = pprint.PrettyPrinter()

contract_id, contract_interface = compiled_sol.popitem()
# pp.pprint(contract_interface['abi'])

# print(web3.eth.accounts)
# print(acct)
web3.personal.unlockAccount(acct, PASSPHRASE)

address = deploy_contract(web3, contract_interface, ARG1, ARG2)
print("Deployed {0} to: {1}\n".format(contract_id.replace('<stdin>:', ''), address))
contractAbi = contract_interface['abi']
pp.pprint(contractAbi)

TypeError: Incorrect argument count.  Expected '2'.  Got '7'

### Event Listening

In [134]:
[{'constant': True,
  'inputs': [{'name': '', 'type': 'uint256'}],
  'name': 'keyList',
  'outputs': [{'name': '', 'type': 'int256'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'maxRounds',
  'outputs': [{'name': '', 'type': 'uint256'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'goalAccuracy',
  'outputs': [{'name': '', 'type': 'uint256'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'vectorLength',
  'outputs': [{'name': '', 'type': 'uint256'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'totalNumData',
  'outputs': [{'name': '', 'type': 'uint256'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'currentRound',
  'outputs': [{'name': '', 'type': 'uint256'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': False,
  'inputs': [{'name': 'update', 'type': 'int256[]'},
             {'name': 'key', 'type': 'int256'},
             {'name': 'numData', 'type': 'int256'}],
  'name': 'sendResponse',
  'outputs': [{'name': '', 'type': 'int256[]'}],
  'payable': False,
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'constant': False,
  'inputs': [{'name': '_clientUpdate', 'type': 'int256[]'},
             {'name': '_numData', 'type': 'uint256'}],
  'name': 'receiveResponse',
  'outputs': [],
  'payable': False,
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'constant': True,
  'inputs': [{'name': '', 'type': 'int256'}, {'name': '', 'type': 'uint256'}],
  'name': 'weights',
  'outputs': [{'name': '', 'type': 'int256'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'numberOfResponses',
  'outputs': [{'name': '', 'type': 'uint256'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'numClients',
  'outputs': [{'name': '', 'type': 'uint256'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'modelType',
  'outputs': [{'name': '', 'type': 'uint256'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'moreThanOne',
  'outputs': [{'name': '', 'type': 'bool'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [{'name': '_vectorLength', 'type': 'int256'},
             {'name': '_keyList', 'type': 'int256[]'},
             {'name': '_goalAccuracy', 'type': 'int256'},
             {'name': '_modelType', 'type': 'int256'},
             {'name': '_maxRounds', 'type': 'int256'},
             {'name': '_initialWeights', 'type': 'int256[]'},
             {'name': '_numClients', 'type': 'int256'}],
  'payable': False,
  'stateMutability': 'nonpayable',
  'type': 'constructor'},
 {'anonymous': False,
  'inputs': [{'indexed': False, 'name': 'client', 'type': 'int256'},
             {'indexed': False, 'name': 'initialWeights', 'type': 'int256[]'}],
  'name': 'ClientSelected',
  'type': 'event'},
 {'anonymous': False,
  'inputs': [{'indexed': False, 'name': 'n', 'type': 'uint256'}],
  'name': 'ResponseReceived',
  'type': 'event'}]

[{'constant': True,
  'inputs': [{'name': '', 'type': 'uint256'}],
  'name': 'keyList',
  'outputs': [{'name': '', 'type': 'int256'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'maxRounds',
  'outputs': [{'name': '', 'type': 'uint256'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'goalAccuracy',
  'outputs': [{'name': '', 'type': 'uint256'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'vectorLength',
  'outputs': [{'name': '', 'type': 'uint256'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'totalNumData',
  'outputs': [{'name': '', 'type': 'uint256'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'currentRound',
  'out

In [135]:
def txn_digest(txn):
    return "Contract address: {0}\nEvent: {1}\nArg: {2}".format(
        txn['address'], txn['event'], txn['args'])

def event_callback(arg):
    print(txn_digest(arg))

contract_obj = web3.eth.contract(
   address=address,
   abi=contractAbi)

# contract_obj.address=contract_obj.address.lower()

pp.pprint(contract_obj.address)

# tx_retval = contract_obj.functions.sendResponse([0, 1, 2], 1, 3).call({'from': TEST_ACCOUNT})
tx_hash = contract_obj.functions.sendResponse([0, 1, 2], 0, 3).transact({'from': TEST_ACCOUNT})
tx_receipt = web3.eth.getTransactionReceipt(tx_hash)
# print(tx_receipt)


# client_selected_event.createFilter({'filter': {'arg1':10}})

# contract_obj.functions.keyList().call({'from': TEST_ACCOUNT})

event_log = contract_obj.events.ResponseReceived().processReceipt(tx_receipt)
# log2 = contract_obj.events.ResponseReceived().processReceipt(tx_receipt)

# pp.pprint(event_log)
event_callback(event_log[0])

# contract_obj.functions.sendResponse([0, 1, 2], 1, 3).transact({'from': TEST_ACCOUNT})
# contract_obj.functions.sendResponse([0, 1, 2], 2, 3).transact({'from': TEST_ACCOUNT})
# contract_obj.functions.inverseScale().transact({'from': TEST_ACCOUNT})

'0x03F642FceB93212d5710c6B5f8B0344150Ae47E1'


ValueError: {'message': 'Attempting to run transaction which calls a contract function, but recipient address 0x03F642FceB93212d5710c6B5f8B0344150Ae47E1 is not a contract address', 'code': -32000, 'data': {'stack': 'n: Attempting to run transaction which calls a contract function, but recipient address 0x03F642FceB93212d5710c6B5f8B0344150Ae47E1 is not a contract address\n    at /usr/local/lib/node_modules/ganache-cli/build/cli.node.js:2:4080985\n    at /usr/local/lib/node_modules/ganache-cli/build/cli.node.js:2:4072925\n    at /usr/local/lib/node_modules/ganache-cli/build/cli.node.js:2:542986\n    at e.exports.c.getCode (/usr/local/lib/node_modules/ganache-cli/build/cli.node.js:2:120503)\n    at /usr/local/lib/node_modules/ganache-cli/build/cli.node.js:2:542936\n    at /usr/local/lib/node_modules/ganache-cli/build/cli.node.js:2:4050264\n    at /usr/local/lib/node_modules/ganache-cli/build/cli.node.js:2:4053943\n    at Object.return (/usr/local/lib/node_modules/ganache-cli/build/cli.nod

In [137]:
# event_filter = contract_obj.eventFilter('ResponseReceived', {'fromBlock': 'latest'})
# print(event_filter.get_all_entries())
# my_filter = web3.eth.filter({"address": address})
# print(my_filter.get_all_entries())
# new_filter = contract_obj.events.ResponseReceived().createFilter(fromBlock = 'latest')
# print(new_filter.get_all_entries())
# block_filter = web3.eth.filter('latest')
# print(block_filter.get_all_entries())

event_signature_hash = web3.sha3(text="ResponseReceived(uint)").hex()
event_filterA = web3.eth.filter({
    "address": address,
    "topics": [event_signature_hash]
    })
event_filterB = web3.eth.filter({
    "address": address.lower(),
    "topics": [event_signature_hash]
    })
print(event_filterA.get_all_entries())
print(event_filterB.get_all_entries())


# new_block_filter = web3.eth.filter('latest')
# for block_hash in new_block_filter.get_all_entries():
#     print("New Block: {}".format(block_hash))

    
# event_filterC = contract_obj.events.ResponseReceived().createFilter(fromBlock = 0)
# event_filterC.get_all_entries()
# print(event_filterC.get_new_entries())

# event_filterD = contract_obj.events.ResponseReceived.createFilter(fromBlock = 0, toBlock = 'latest')
# print(event_filterD.get_all_entries())

# event_filter_old = contract_obj.eventFilter('EventEmit', {'fromBlock': 0,'toBlock': 'latest'})
# event_filter_old.get_all_entries()

[]
[]


In [123]:

def wait_on_tx_receipt(tx_hash):
    start_time = time.time()
    while True:
        if start_time + 60 < time.time():
            raise TimeoutError("Timeout occurred waiting for tx receipt")
        if w3.eth.getTransactionReceipt(tx_hash):
            return w3.eth.getTransactionReceipt(tx_hash)

# Create log filter instance
_filter = contract_obj.events.EventEmit.createFilter(fromBlock=0)

# Event emitting transaction
tx_hash = contract_obj.functions.sendResponse([0, 1, 2], 0, 3).transact({'from': TEST_ACCOUNT})
# Wait for transaction to be mined
receipt = wait_on_tx_receipt(tx_hash)

# Poll filter
print(_filter.get_new_entries())

AttributeError: 'ContractEvents' object has no attribute 'EventEmit'

In [128]:
print(address)
print(w3.toChecksumAddress(address))

w3.toChecksumAddress(address) == address

0x9Ab9b0689Fe27379322357F0Cec27DB7797D8679
0x9Ab9b0689Fe27379322357F0Cec27DB7797D8679


True